In [1]:
import os
import torch
from utils import get_vocabs_dep_parser
torch.manual_seed(1)

In [2]:
data_dir = 'data_new'
path_train = os.path.join(data_dir, 'train.labeled')
path_test = os.path.join(data_dir, 'test.labeled')
paths_list = [path_train, path_test]
word_dict, pos_dict = get_vocabs_dep_parser(paths_list)

In [3]:
from data_handling import DepDataset
from torch.utils.data.dataloader import DataLoader
print("path_train -", path_train)
print("path_test -", path_test)
BATCH_SIZE = 1

train = DepDataset(word_dict, pos_dict, data_dir, 'train', padding=False)
train_dataloader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
test = DepDataset(word_dict, pos_dict, data_dir, 'test', padding=False)
test_dataloader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)

path_train - data_new/train.labeled
path_test - data_new/test.labeled
idx_pos_mappings - [0, 1, 2, '#', '$', "''", '(', ')', ',', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
pos_idx_mappings - {'<pad>': 0, '<unk>': 1, '<root>': 2, '#': 3, '$': 4, "''": 5, '(': 6, ')': 7, ',': 8, '.': 9, ':': 10, 'CC': 11, 'CD': 12, 'DT': 13, 'EX': 14, 'FW': 15, 'IN': 16, 'JJ': 17, 'JJR': 18, 'JJS': 19, 'LS': 20, 'MD': 21, 'NN': 22, 'NNP': 23, 'NNPS': 24, 'NNS': 25, 'PDT': 26, 'POS': 27, 'PRP': 28, 'PRP$': 29, 'RB': 30, 'RBR': 31, 'RBS': 32, 'RP': 33, 'SYM': 34, 'TO': 35, 'UH': 36, 'VB': 37, 'VBD': 38, 'VBG': 39, 'VBN': 40, 'VBP': 41, 'VBZ': 42, 'WDT': 43, 'WP': 44, 'WP$': 45, 'WRB': 46, '``': 47}
idx_pos_mappings - [0, 1, 2, '#', '$', "''", '(', ')', ',', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', '

In [4]:
print("Number of Train Sentences ", len(train))
print("Number of Test Sentences ",len(test))

Number of Train Sentences  5000
Number of Test Sentences  1000


In [5]:
import torch
from torch import nn
import torch.optim as optim
from model import DnnSepParser
from trainer import Trainer
from loss import NllLoss
from utils import IGNORE_IDX

WORD_EMBEDDING_DIM = 100
TAG_EMBEDDING_DIM = 25
NUM_LAYERS = 2
word_vocab_size = len(train.word_idx_mappings)
tag_vocab_size = len(train.pos_idx_mappings)
max_sentence_len = max(train.max_seq_len, test.max_seq_len)
model = DnnSepParser(WORD_EMBEDDING_DIM, TAG_EMBEDDING_DIM, NUM_LAYERS, word_vocab_size, tag_vocab_size, max_sentence_len)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

if use_cuda:
    model.cuda()

#TODO: use our function with ignore index
# loss_function = nn.NLLLoss(ignore_index=IGNORE_IDX)
loss_function = nn.NLLLoss()
# loss_function = NllLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
acumulate_grad_steps = 50 # This is the actual batch_size, while we officially use batch_size=1
trainer = Trainer(model, optimizer, loss_function, device)

In [ ]:
torch.manual_seed(1)
EPOCHS = 100
acumulate_grad_steps = 50
len_train = len(train)
len_test = len(test)
trainer.train_dep_parser(EPOCHS, train_dataloader, test_dataloader, acumulate_grad_steps, len_train, len_test, early_stopping=5)

Epoch: 1 | Training Loss: 0.0670 | Training accuracy: 10.046% | Test Loss: 0.0647 | Test accuracy: 9.546% | Epoch Time: 33.54 secs
saving model
Epoch: 2 | Training Loss: 0.0638 | Training accuracy: 11.539% | Test Loss: 0.0648 | Test accuracy: 9.874% | Epoch Time: 39.06 secs
saving model
Epoch: 3 | Training Loss: 0.0628 | Training accuracy: 12.630% | Test Loss: 0.0649 | Test accuracy: 10.045% | Epoch Time: 33.96 secs
saving model
Epoch: 4 | Training Loss: 0.0618 | Training accuracy: 14.113% | Test Loss: 0.0657 | Test accuracy: 10.039% | Epoch Time: 32.97 secs
Epoch: 5 | Training Loss: 0.0606 | Training accuracy: 15.257% | Test Loss: 0.0664 | Test accuracy: 10.133% | Epoch Time: 35.20 secs
saving model
Epoch: 6 | Training Loss: 0.0594 | Training accuracy: 17.058% | Test Loss: 0.0680 | Test accuracy: 9.899% | Epoch Time: 32.69 secs
Epoch: 7 | Training Loss: 0.0582 | Training accuracy: 17.766% | Test Loss: 0.0696 | Test accuracy: 9.690% | Epoch Time: 32.80 secs
Epoch: 8 | Training Loss: 0.